### Applications
Use sqlalchemy and flask to create a connection to an existing sqlite database

In [ ]:
app = Flask(__name__)
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///db/bellybutton.sqlite"
db = SQLAlchemy(app)

Use pandas to perform sql queries to return jsonified sample names

In [ ]:
@app.route("/names")
def names():
    """Return a list of sample names."""

    # Use Pandas to perform the sql query
    stmt = db.session.query(Samples).statement
    df = pd.read_sql_query(stmt, db.session.bind)

    # Return a list of the column names (sample names)
    return jsonify(list(df.columns)[2:])

Use flask to serve up the jsonified sample metadata

In [ ]:
@app.route("/metadata/<sample>")
def sample_metadata(sample):
    """Return the MetaData for a given sample."""
    sel = [
        Samples_Metadata.sample,
        Samples_Metadata.ETHNICITY,
        Samples_Metadata.GENDER,
        Samples_Metadata.AGE,
        Samples_Metadata.LOCATION,
        Samples_Metadata.BBTYPE,
        Samples_Metadata.WFREQ,
    ]

    results = db.session.query(*sel).filter(Samples_Metadata.sample == sample).all()

    # Create a dictionary entry for each row of metadata information
    sample_metadata = {}
    for result in results:
        sample_metadata["sample"] = result[0]
        sample_metadata["ETHNICITY"] = result[1]
        sample_metadata["GENDER"] = result[2]
        sample_metadata["AGE"] = result[3]
        sample_metadata["LOCATION"] = result[4]
        sample_metadata["BBTYPE"] = result[5]
        sample_metadata["WFREQ"] = result[6]

    print(sample_metadata)
    return jsonify(sample_metadata)

Use flask to serve up the jsonified sample data containing otu_ids, sample_values and otu_labels

## Data Inspection

In [1]:
# Imports & Dependencies
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

In [4]:

# Establishing connection to database
dbfile = "belly_button_biodiversity.sqlite"
engine = create_engine(f"sqlite:///db/{dbfile}")

In [5]:
# Preparing and reflecting data
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

['otu', 'samples', 'samples_metadata']

In [6]:
# Creating session & variables to query later
session = Session(engine)
Samples = Base.classes.samples
OTU = Base.classes.otu
metadata = Base.classes.samples_metadata

In [7]:
# Extracting list of sample names
samples_stmt = session.query(Samples).statement
samples_df = pd.read_sql_query(samples_stmt, session.bind)
samples_df.set_index("otu_id", inplace = True)
sample_names = list(samples_df.columns)
print(sample_names)

['BB_940', 'BB_941', 'BB_943', 'BB_944', 'BB_945', 'BB_946', 'BB_947', 'BB_948', 'BB_949', 'BB_950', 'BB_952', 'BB_953', 'BB_954', 'BB_955', 'BB_956', 'BB_958', 'BB_959', 'BB_960', 'BB_961', 'BB_962', 'BB_963', 'BB_964', 'BB_966', 'BB_967', 'BB_968', 'BB_969', 'BB_970', 'BB_971', 'BB_972', 'BB_973', 'BB_974', 'BB_975', 'BB_978', 'BB_1233', 'BB_1234', 'BB_1235', 'BB_1236', 'BB_1237', 'BB_1238', 'BB_1242', 'BB_1243', 'BB_1246', 'BB_1253', 'BB_1254', 'BB_1258', 'BB_1259', 'BB_1260', 'BB_1264', 'BB_1265', 'BB_1273', 'BB_1275', 'BB_1276', 'BB_1277', 'BB_1278', 'BB_1279', 'BB_1280', 'BB_1281', 'BB_1282', 'BB_1283', 'BB_1284', 'BB_1285', 'BB_1286', 'BB_1287', 'BB_1288', 'BB_1289', 'BB_1290', 'BB_1291', 'BB_1292', 'BB_1293', 'BB_1294', 'BB_1295', 'BB_1296', 'BB_1297', 'BB_1298', 'BB_1308', 'BB_1309', 'BB_1310', 'BB_1374', 'BB_1415', 'BB_1439', 'BB_1441', 'BB_1443', 'BB_1486', 'BB_1487', 'BB_1489', 'BB_1490', 'BB_1491', 'BB_1494', 'BB_1495', 'BB_1497', 'BB_1499', 'BB_1500', 'BB_1501', 'BB_1502'

In [8]:
print(samples_df)

        BB_940  BB_941  BB_943  BB_944  BB_945  BB_946  BB_947  BB_948  \
otu_id                                                                   
1            0       0       0       0       0       0       0       0   
2            0       0       0       0       0       0       0       0   
3            0       0       0       0       0       0       0       0   
4            0       0       0       0       0       0       0       0   
5            0       0       0       0       0       0       0       0   
6            0       0       0       0       0       0       0       0   
7            0       0       0       0       0       0       0       0   
8            0       0       0       0       0       0       0       0   
9            0       0       0       0       0       1       0       0   
10           0       0       0       0       0       0       0       0   
11           1       0       0       0       0       0       0       1   
12           0       0       0       0

In [9]:
# Extracting OTU descriptions
otu_stmt = session.query(OTU).statement
otu_df = pd.read_sql_query(otu_stmt, session.bind)
otu_df.set_index("otu_id", inplace=True)
otu_descriptions = list(otu_df.lowest_taxonomic_unit_found)
print(otu_descriptions)

['Archaea;Euryarchaeota;Halobacteria;Halobacteriales;Halobacteriaceae;Halococcus', 'Archaea;Euryarchaeota;Halobacteria;Halobacteriales;Halobacteriaceae;Halococcus', 'Archaea;Euryarchaeota;Halobacteria;Halobacteriales;Halobacteriaceae;Natronorubrum', 'Archaea;Euryarchaeota;Methanobacteria;Methanobacteriales;Methanobacteriaceae;Methanobrevibacter', 'Archaea;Euryarchaeota;Methanobacteria;Methanobacteriales;Methanobacteriaceae;Methanobrevibacter', 'Archaea;Euryarchaeota;Methanobacteria;Methanobacteriales;Methanobacteriaceae;Methanobrevibacter', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria

In [10]:
# Extracting metadata for each sample
metadata_stmt = session.query(metadata).statement
metadata_df = pd.read_sql_query(metadata_stmt, session.bind)
metadata_df.head()

,SAMPLEID,EVENT,ETHNICITY,GENDER,AGE,WFREQ,BBTYPE,LOCATION,COUNTRY012,ZIP012,...,DOG,CAT,IMPSURFACE013,NPP013,MMAXTEMP013,PFC013,IMPSURFACE1319,NPP1319,MMAXTEMP1319,PFC1319
0,940,BellyButtonsScienceOnline,Caucasian,F,24,2.0,I,Beaufort/NC,usa,22306,...,no,no,8852,37.172222,54.5,0.0,1,0.0,33.990002,25.5
1,941,0,Caucasian/Midleastern,F,34,1.0,I,Chicago/IL,0,0,...,0,0,0,0.000000,0.0,0.0,0,0.0,0.000000,0.0
2,943,BellyButtonsScienceOnline,Caucasian,F,49,1.0,I,Omaha/NE,0,0,...,0,0,0,0.000000,0.0,0.0,0,0.0,0.000000,0.0
3,944,BellyButtonsScienceOnline,European,M,44,1.0,I,NewHaven/CT,usa,7079,...,no,yes,0,35.816666,16.0,0.0,0,6567.0,32.403332,28.5
4,945,BellyButtonsScienceOnline,Caucasian,F,48,1.0,I,Philidelphia/PA,usa,84404,...,no,no,0,37.783333,4.0,0.0,0,5613.0,33.634445,24.0


In [11]:
# Extracting otu_id and sample values for given sample
test_sample = "BB_940"
samples_statement = session.query(Samples).statement
df = pd.read_sql_query(samples_statement, session.bind)
df = df[df[test_sample] > 1]
df = df.sort_values(by=test_sample, ascending=0)

df

,otu_id,BB_940,BB_941,BB_943,BB_944,BB_945,BB_946,BB_947,BB_948,BB_949,...,BB_1562,BB_1563,BB_1564,BB_1572,BB_1573,BB_1574,BB_1576,BB_1577,BB_1581,BB_1601
1166,1167,163,24,0,0,32,217,27,18,0,...,0,0,0,93,0,14,0,0,0,73
2858,2859,126,21,0,0,40,195,200,47,0,...,0,0,0,19,0,14,0,0,0,0
481,482,113,8,0,0,9,28,3,50,0,...,0,0,0,67,0,29,0,0,0,0
2263,2264,78,4,0,0,0,12,1,18,0,...,0,0,0,4,0,0,0,0,0,0
40,41,71,4,0,3,14,153,13,10,0,...,0,0,0,12,0,0,0,0,0,0
1188,1189,51,0,0,0,5,26,0,20,0,...,0,0,0,17,0,2,0,0,0,0
351,352,50,3,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
188,189,47,2,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2317,2318,40,1,0,0,7,31,6,6,0,...,0,0,0,0,0,2,0,0,0,0
1976,1977,40,2,0,0,0,9,1,29,0,...,0,0,0,23,0,1,0,0,0,0
